In [107]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import * 
from fastai.vision import *
from fastai.tabular.all import *
from numpy import random
import pandas as pd
import numpy as np
import array as arr

train_df = pd.read_csv("data/03-05-2021_world.csv")
train_df.head().T
#display_all(pops_raw.head().transpose())

# take the incidence rate / testing rate for the incidence proportion (risk)
# note: ~50% people wear masks.

,0,1,2,3,4
FIPS,NaN,NaN,NaN,NaN,NaN
Admin2,NaN,NaN,NaN,NaN,NaN
Province_State,NaN,NaN,NaN,NaN,NaN
Country_Region,Afghanistan,Albania,Algeria,Andorra,Angola
Last_Update,2021-03-06 04:23:41,2021-03-06 04:23:41,2021-03-06 04:23:41,2021-03-06 04:23:41,2021-03-06 04:23:41
Lat,33.93911,41.1533,28.0339,42.5063,-11.2027
Long_,67.709953,20.1683,1.6596,1.5218,17.8739
Confirmed,55827,111301,113948,10998,21026
Deaths,2449,1897,3007,112,511
Recovered,49362,73610,78824,10584,19624


In [83]:
# take care of missing values
train_df['Incident_Rate'].fillna(0, inplace=True)
display_all(train_df.isnull().sum().sort_index()/len(train_df))
#train_df.fillna(0, inplace=True)

Active                 0.001005
Admin2                 0.178850
Case_Fatality_Ratio    0.009797
Combined_Key           0.000000
Confirmed              0.000000
Country_Region         0.000000
Deaths                 0.000000
FIPS                   0.180106
Incident_Rate          0.000000
Last_Update            0.000000
Lat                    0.022105
Long_                  0.022105
Province_State         0.042954
Recovered              0.000000
dtype: float64

In [84]:
train_df.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-03-06 04:23:41,33.93911,67.709953,55827,2449,49362,4016.0,Afghanistan,143.409656,4.386766
1,NaN,NaN,NaN,Albania,2021-03-06 04:23:41,41.15330,20.168300,111301,1897,73610,35794.0,Albania,3867.572451,1.704387
2,NaN,NaN,NaN,Algeria,2021-03-06 04:23:41,28.03390,1.659600,113948,3007,78824,32117.0,Algeria,259.852428,2.638923
3,NaN,NaN,NaN,Andorra,2021-03-06 04:23:41,42.50630,1.521800,10998,112,10584,302.0,Andorra,14234.129295,1.018367
4,NaN,NaN,NaN,Angola,2021-03-06 04:23:41,-11.20270,17.873900,21026,511,19624,891.0,Angola,63.974407,2.430324


In [85]:
#idx = np.random.permutation(range(n))[:2000]
#idx.sort()
#small_train_df = train_df.iloc[idx[:1000]]
#small_test_df = train_df.iloc[idx[1000:]]
small_cont_vars = ['Active', 'Confirmed', 'Deaths', 'Recovered', 'Case_Fatality_Ratio']
small_cat_vars =  ['Province_State', 'Last_Update', 'Admin2', 'FIPS']
#small_train_df = small_train_df[small_cat_vars + small_cont_vars + ['Incident_Rate']]
#small_test_df = small_test_df[small_cat_vars + small_cont_vars + ['Incident_Rate']]
procs = [Categorify, FillMissing, Normalize]

In [86]:
#small_train_df.head()

In [87]:
#small_test_df.head()

In [88]:
dls = TabularDataLoaders.from_df(train_df, path = "data/03-05-2021_world.csv", procs=procs, cat_names=small_cat_vars, cont_names=small_cont_vars, y_names = 'Incident_Rate', 
                                  valid_idx=list(range(800,1000)), bs=64)

In [89]:
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df))

In [90]:
to = TabularPandas(train_df, procs=procs,
                   cat_names = small_cat_vars,
                   cont_names = small_cont_vars,
                   y_names = 'Incident_Rate',
                   splits=splits)

In [91]:
to.xs.iloc[:3]

,Province_State,Last_Update,Admin2,FIPS,Active_na,Case_Fatality_Ratio_na,Active,Confirmed,Deaths,Recovered,Case_Fatality_Ratio
1235,198,7,292,456,1,1,-0.095959,-0.176724,-0.169059,-0.151451,-0.345793
81,339,7,0,0,1,1,-0.123719,-0.185743,-0.168273,-0.139457,-0.036747
255,100,7,0,0,1,1,-0.096226,1.796915,0.834625,2.508459,-0.242196


In [92]:
dls = to.dataloaders(bs=64)

In [93]:
dls.show_batch()

,Province_State,Last_Update,Admin2,FIPS,Active_na,Case_Fatality_Ratio_na,Active,Confirmed,Deaths,Recovered,Case_Fatality_Ratio,Incident_Rate
0,Mississippi,2021-03-06 04:23:41,Issaquena,28055.0,False,False,161.000059,166.999541,6.000007,0.000286,3.592814e+00,12584.777344
1,Illinois,2021-03-06 04:23:41,Mason,17125.0,False,False,1245.000144,1286.000645,41.000024,0.000286,3.188180e+00,9626.468750
2,Quintana Roo,2021-03-06 04:23:41,#na#,#na#,False,False,17927.999838,20346.000406,2417.999952,0.000286,1.188440e+01,1180.669922
3,Hyogo,2021-03-06 04:23:41,#na#,#na#,False,False,462.999730,18190.999724,555.000000,17173.000001,3.050959e+00,332.791199
4,North Carolina,2021-03-06 04:23:41,Burke,37023.0,False,False,9133.999957,9275.001143,140.999976,0.000286,1.520216e+00,10250.317383
5,Oklahoma,2021-03-06 04:23:41,Craig,40035.0,False,False,1886.000331,1897.001090,10.999993,0.000286,5.798629e-01,13413.944336
6,Indiana,2021-03-06 04:23:41,Franklin,18047.0,False,False,1561.999791,1597.000184,34.999984,0.000286,2.191609e+00,7017.312500
7,Texas,2021-03-06 04:23:41,McLennan,48309.0,False,False,24848.000138,25291.000244,442.999996,0.000286,1.751611e+00,9855.313477
8,Brussels,2021-03-06 04:23:41,#na#,#na#,False,False,96994.997084,96994.999504,0.000023,0.000286,1.001781e-07,8025.786621
9,Louisiana,2021-03-06 04:23:41,Webster,22119.0,False,False,3842.999630,3940.000009,96.999983,0.000286,2.461929e+00,10276.473633


In [94]:
learn = tabular_learner(dls, metrics=accuracy)

In [95]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,72552696.000000,78157440.000000,0.021357,00:00


In [96]:
learn.show_results()

,Province_State,Last_Update,Admin2,FIPS,Active_na,Case_Fatality_Ratio_na,Active,Confirmed,Deaths,Recovered,Case_Fatality_Ratio,Incident_Rate,Incident_Rate_pred
0,494.0,7.0,1764.0,0.0,1.0,1.0,-0.106573,-0.183401,-0.163292,-0.151451,0.153057,10154.895508,-0.108929
1,318.0,7.0,588.0,0.0,1.0,1.0,-0.121328,-0.193163,-0.175088,-0.151451,-0.390045,9768.095703,-0.106793
2,198.0,7.0,370.0,0.0,1.0,1.0,-0.114252,-0.188555,-0.172467,-0.151451,-0.251763,11466.422852,-0.122945
3,313.0,7.0,122.0,0.0,1.0,1.0,-0.113540,-0.188028,-0.169583,-0.151451,0.015822,6721.494141,-0.115157
4,523.0,7.0,1769.0,0.0,1.0,1.0,0.766227,0.380489,-0.086227,-0.151451,-0.491459,14212.043945,-0.113874
5,154.0,7.0,1646.0,0.0,1.0,1.0,-0.095356,-0.176032,-0.156477,-0.151451,0.113538,11483.124023,-0.091656
6,345.0,7.0,620.0,0.0,1.0,1.0,-0.103005,-0.181332,-0.172467,-0.151451,-0.436315,4282.115723,-0.083677
7,318.0,7.0,1193.0,0.0,1.0,1.0,-0.121090,-0.192973,-0.173515,-0.151451,0.155759,6647.960449,-0.104321
8,76.0,7.0,1164.0,0.0,1.0,1.0,0.291443,0.074027,-0.089373,-0.151451,-0.373475,9761.992188,-0.112788


In [97]:
preds, targs = learn.get_preds()

In [98]:
cum_prob = (1.8+2.1)/100; cum_prob # probability according to: https://www.acpjournals.org/doi/10.7326/M20-6817

0.03900000000000001

In [99]:
prob_mask = 1.8/3.9; prob_mask

0.46153846153846156

In [100]:
prob_nomask = 2.1/3.9; prob_nomask

0.5384615384615385

In [101]:
irlen = len(train_df.Incident_Rate);

In [125]:
train_df.Incident_Rate
infected_mask_list = [0] * irlen

In [126]:
np.int64(np.ceil(train_df.Incident_Rate[1]))


3868

In [136]:
for i in range(irlen):
    i_rate = np.int64(np.ceil(train_df.Incident_Rate[i]))
    imask_counter = 0
    for j in range(i_rate):
        x = random.rand()
        if x <= prob_mask:
            imask_counter += 1
    infected_mask_list[i] = imask_counter

In [137]:
display(infected_mask_list) # display the array, sorted by region/country alphabetically. each entry is how many cases were infected while wearing masks

[69,
 1765,
 121,
 6661,
 25,
 373,
 2174,
 2686,
 12,
 28,
 21,
 11,
 16,
 22,
 129,
 17,
 2409,
 1096,
 981,
 3367,
 131,
 523,
 1444,
 2445,
 3688,
 2600,
 2450,
 4163,
 4364,
 2056,
 3658,
 3753,
 0,
 3473,
 2706,
 1432,
 25,
 54,
 1010,
 1899,
 611,
 3092,
 1792,
 4669,
 3604,
 2198,
 2237,
 4646,
 3816,
 2745,
 1457,
 3383,
 3098,
 1961,
 2033,
 2595,
 2684,
 1462,
 2450,
 2252,
 2703,
 1603,
 4063,
 6343,
 4580,
 2141,
 3047,
 3389,
 19,
 1716,
 25,
 118,
 11,
 1349,
 3,
 71,
 1401,
 796,
 0,
 0,
 1067,
 81,
 74,
 46,
 73,
 444,
 1019,
 47,
 1528,
 0,
 1183,
 79,
 35,
 16,
 2921,
 2019,
 3153,
 1978,
 1388,
 2308,
 1294,
 2924,
 2468,
 6069,
 1954,
 2400,
 1657,
 1569,
 3605,
 0,
 1367,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 4,
 1,
 64,
 52,
 1,
 1,
 0,
 3,
 0,
 1,
 1,
 2,
 0,
 1,
 1,
 4,
 1,
 2,
 2,
 0,
 0,
 3,
 0,
 2,
 3234,
 2458,
 961,
 2321,
 1505,
 1811,
 2181,
 4116,
 1963,
 1336,
 868,
 1617,
 554,
 1024,
 1697,
 1194,
 1228,
 2117,
 903,
 1139,
 1893,
 1345,
 1580,
 1